# Agentic AI with a decision process to select a 'researcher' or 'journalist' as agents using crewAI 

# Instructions:

- Go to the **Assessments** tab in iCollege and click on **Assignments** to find the **Homework 2** assignment and further instructions to the assignment.

- Make a copy of this homework notebook and rename the copy using your GSU username as suffix:

    **AgenticAI_RAG_Homework2_\<YOUR GSU USERNAME\>.ipynb**

- Use the **MSA8700 Python 3.12** kernel when you execute your noterbnook cells! This kernel has all the necessary Python packages already installed. You do not have to install any packages. Do **NOT** use any of the other kernels. 

- Work with your renamed copy to create your solution and all the necessary .txt files that will be generated and submitted with your notebook. 

- Complete all cells in the notebook i.e. replace missing fields (abbreviated with **'...'** in the code cells below) with your code cells, run the notebook and create the 6 necessary .txt files. 

- Run the tests to confirm that your .txt files were generated. Finally, submit your .txt files and your executed .ipynb notebook by running the two submission steps at the very end of the notebook.

# Important Note:

- When running the 'researcher' branch you might notice that the **`pdf_search_tool` might fail**, i.e. a response cannot be generated fron the PDF file and you get an error message.

- When that occurs **delete the database folder db/** (e.g. with **rm -rf db** on the command line in a terminal window; you might have to restart or shutdown the kernel before you can do that) that has been generated and re-run the cells. The database folder db/ will be re-created and `pdf_search_tool` should return a result with showing an error.

In [1]:
# Do not change the content of this cell. Execute this cell first, and everytime after you restarted the kernel.
%reload_ext autoreload
%autoreload 2

In [3]:
import os
from pprint import pprint
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai_tools import PDFSearchTool

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/anaconda3/envs/conda-msa8700/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/anaconda3/envs/conda-msa8700/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.Skip

### Load the API keys

In [4]:
# Load your API keys here and create your environment variables.

# IMPORTANT: Do not directly copy your API keys into your notebooks and submit them with your solution!

OPENAI_API_KEY =''
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

GEMINI_API_KEY = ''
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

SERPER_API_KEY = ''
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [5]:
GROQ_API_KEY = ''
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

### Load the PDF document

Upload NIPS 2017 paper and store locally in **data/** folder.

Source: 

**Advances in Neural Information Processing Systems 30 (NIPS 2017)**. Edited by: I. Guyon and U. Von Luxburg and S. Bengio and H. Wallach and R. Fergus and S. Vishwanathan and R. Garnett. ISBN: 9781510860964. https://proceedings.neurips.cc/paper_files/paper/2017

**Attention is All you Need** Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez, Łukasz Kaiser, Illia Polosukhin. https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf

In [6]:
import requests

pdf_url = 'https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf'
response = requests.get(pdf_url)

with open('data/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'wb') as file:
    file.write(response.content)

### Initialize LLMs

In [7]:
# Upgrade the langchain-openai package first if needed:
#!pip install -U langchain-openai

from langchain_openai import ChatOpenAI

crew_llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-3.5-turbo",  # or "gpt-4-1106-preview" if you have access
    max_tokens=1024,
    temperature=0.7
)

In [8]:
# Alternative:
# Pick a Groq chat model with API key, max_tokens and temparature parameter values:



### Define Tools

In [9]:
from crewai_tools import PDFSearchTool

pdf_search_tool = PDFSearchTool(
    pdf='data/NIPS-2017-attention-is-all-you-need-Paper.pdf',
    config=dict(
        llm=dict(
            provider="openai",
            config=dict(
                model="gpt-4",
                temperature=0.7,
                top_p=1,
                stream=False
            )
        ),
        embedder=dict(
            provider="openai",
            config=dict(
                model="text-embedding-3-small"
                # Do not include task_type or title here
            )
        )
    )
)


In [10]:
# custom-defined tool using the @tool decrorator:
@tool("Router Function")
def router_tool(question: str) -> str:
  """Router Function"""
  if 'self-attention' in question:
    return 'researcher'
  else:
    return 'journalist'

### Create decision router

In [11]:
# Create a decision router function based on an input 'query':

# The `decide_hiring()` function shall make a binary decision and return a string. 
# This function shall either return the value 'researcher' or the value 'journalist' 
# depending on whether a certain query can be answered with the content in a PDF file 
# or has to be answered via a web search with web scraping task.

# An agent named 'router' shall execute a task using the tool `pdf_search_tool` to determine whether
# the query can be answered with the PDF file content or has to be answered via a web search with web scraping activity.

# Enter definitions for the variables 'role', 'goal' and 'backstory' of the Agent 'router'.
# Enter definitions for the variables 'description', 'expected_output', 'agent', and 'tools' of the Task 'router_task'.
# Enter definitions for the variables 'agents' and 'tasks' in the Crew class 'crew'. 
# Finally, define the 'inputs' dictionary in order to execute `result = crew.kickoff(inputs={...})` to generate an output string 'result.raw'

# Note: You might have to experiment with different definitions an experiments to get a well-performing agentic workflow.

from crewai import Agent, Task, Crew

def decide_hiring(query):
    """Router function to determine if we can answer from the existing PDF file"""

    router = Agent(
        role='router',
        goal='Decide if a query can be answered from the PDF document',
        backstory=(
            "You are a smart router that determines whether a given user question "
            "can be answered using the PDF content or requires external web search."
        ),
        verbose=True,
        allow_delegation=False,
        llm=crew_llm,  # assuming crew_llm is already initialized
    )

    router_task = Task(
        description=f"Given the question: '{query}', determine if it can be answered using only the provided PDF document.",
        expected_output="Return 'yes' if the PDF content contains enough information to answer the question. Return 'no' otherwise.",
        agent=router,
        tools=[pdf_search_tool],  # assuming pdf_search_tool is already initialized
    )

    crew = Crew(
        agents=[router],
        tasks=[router_task],
        verbose=1,
        memory=False,
    )

    result = crew.kickoff(inputs={"query": query})
    return result.raw


### Create branch workflows with agents, tasks and crews

#### Create the 'researcher' branch in the process: 

In [12]:
# For the 'researcher' branch, we want to define a function `hire_researcher(query)` that is executed 
# if the 'router' agent determines that the 'query' can be answered by the content avaialble in the PDF file. 

# Define an Agent 'researcher', Task 'researcher_task' and a Crew that will use the tool `pdf_search_tool` to
# provide a response to the query from the PDF tile.

# Similar to the 'router' workflow above:
# Enter definitions for the variables 'role', 'goal' and 'backstory' of the Agent 'researcher'. 
# For the 'role' you can specify 'Expert Data Analyst'.

# Enter definitions for the variables 'description', 'expected_output', 'agent', and 'tools' of the Task 'researcher_task'.
# In the 'description' argument refer to the '{topic}' variable as placeholder for the input query.  

# Enter definitions for the variables 'agents' and 'tasks' in the Crew class 'crew'. 

# By executing `result = crew.kickoff(inputs={"topic": query})` the response from searching the PDF file 
# is generated for the input 'query' by the agentic workflow. 

# Adding the PDF Searching Agent
def hire_researcher(query):
    # Researcher agent
    researcher = Agent(
        role="Expert Data Analyst",
        goal="Answer user questions using information found in a PDF document.",
        backstory=(
            "You are an expert data analyst skilled in navigating and understanding academic research papers. "
            "You help users by finding accurate, relevant information in the provided PDF document to answer their queries clearly."
        ),
        allow_delegation=False,
        verbose=True,
        llm=crew_llm  # assumed to be initialized as ChatOpenAI or compatible LLM
    )

    # Task for the researcher
    researcher_task = Task(
        description="Use the PDF document to answer the following question: '{topic}'",
        expected_output="A concise and accurate answer to the query based solely on the PDF content.",
        tools=[pdf_search_tool],  # assumed to be set up with the PDF path
        agent=researcher
    )

    # Create the Crew
    crew = Crew(
        agents=[researcher],
        tasks=[researcher_task],
        memory=False,
        verbose=1
    )

    print("The researcher was hired...")
    result = crew.kickoff(inputs={"topic": query})
    return result.raw


#### Create the 'journalist' branch in the process: 

In [13]:
# For the 'journalist' branch, alternatively, a function `hire_journalist(query)` is executed 
# if the 'router' agent determines that the 'query' cannot be answered by the content available in the PDF file 
# but needs to be answered via a web searching and web scraping activity. 

# See the Agent definitions of 'web_search_agent' and 'web_scraper_agent'.

# Enter definitions for the variables 'description', 'expected_output', 'agent', and 'tools' 
# of the 'search_task' and 'scraping_task'.

# In the 'description' argument refer to the '{topic}' variable as placeholder for the input query.  

# Enter definitions for the variables 'agents' and 'tasks' in the Crew class 'crew'. 

# By executing `result = crew.kickoff(inputs={"topic": query})` the response 
# from searching and scraping the web.

# Note: Focus on the content scraped from the most relevant searched website with respect to the inout 'query' 
# in order to limit the number of iterations in the web searching and web scraping process. 

# Adding the Web Searching and Scraping Agent
def run_web_search_scraping(query):
    """Setup the web scraping agent and related components"""
    search_tool = SerperDevTool()  # Tool for performing web searches
    scrape_website = ScrapeWebsiteTool()  # Tool for extracting data from websites
    
    # Define the web search agent
    web_search_agent = Agent(
        role="Expert Web Search Agent",
        goal="Identify and retrieve relevant web data for user queries",
        backstory="An expert in identifying valuable web sources for the user's needs",
        allow_delegation=False,
        verbose=True,
        llm=crew_llm
    )
    
    # Define the web scraping agent
    web_scraper_agent = Agent(
        role="Expert Web Scraper Agent",
        goal="Extract and analyze content from specific web page identified by the search agent",
        backstory="A highly skilled web scraper, capable of analyzing and summarizing website content accurately",
        allow_delegation=False,
        verbose=True,
        llm=crew_llm
    )
    
    # Define the web search task
    search_task = Task(
        description="Search the web and identify the most relevant source to answer the following query: '{topic}'",
        expected_output="The URL of the most relevant and trustworthy web page",
        tools=[search_tool],
        agent=web_search_agent,
    )
    
    # Define the web scraping task
    scraping_task = Task(
        description="Extract and summarize key insights from the webpage related to: '{topic}'",
        expected_output="A well-structured answer based on the content of the selected webpage",
        tools=[scrape_website],
        agent=web_scraper_agent,
    )
    
    # Define the crew to manage agents and tasks
    crew = Crew(
        agents=[web_search_agent, web_scraper_agent],
        tasks=[search_task, scraping_task],
        verbose=1,
        memory=False,
    )
    
    print("The journalist was hired ...")
    result = crew.kickoff(inputs={"topic": query})
    return result.raw
def generate_journalist_answer(context, query):
    """Generate final answer using LLM for RAG"""
    messages = [
        (
            "system",
            "You are an investigative journalist skilled in distilling information from web research into clear and accurate answers.",
        ),
        ("system", f"Context: {context}"),
        ("human", query),
    ]
    response = crew_llm.invoke(messages)
    return response.content


In [14]:
def hire_journalist(query): 
    # Retrieve context from web scraping
    context = run_web_search_scraping(query)
    # Generate final answer via RAG
    answer = generate_journalist_answer(context, query)
    return answer

#### Define process

In [15]:
# Create the necessary process flow with the functions `decide_hiring`, `hire_researcher`, `hire_journalist` 
# defined above to create the agentic workflow as described above.

# Replace the dots below (...) with your code insertions:

def process_query(query):
    """Main function to process user query"""
    print(f"Processing query: {query}")

    # Step 1: Decide which team will be hired based on the question (input query)
    decision = decide_hiring(query).lower()

    # Step 2: Generate final answer based on whether 'researcher' or 'journalist' branch was selected
    if "yes" in decision:
        # Researcher branch (answer can be found in the PDF)
        context = hire_researcher(query)
        answer = generate_journalist_answer(context, query)
    else:
        # Journalist branch (answer needs web search + scraping)
        context = run_web_search_scraping(query)
        answer = generate_journalist_answer(context, query)

    return answer


#### Execute process with query

In [16]:
# Create three different appropriate questions (input query) to test the execution of the 'researcher' branch
# where the 'researcher' agent extracts an answer for the question from the uploaded PDF file:

# Example: "How did the self-attention mechanism evolve in large language models?"

# Choose a different question as input!

In [36]:
# Pick your first 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
# Pick your first 'question'
question = "What is the transformer architecture introduced in the 'Attention is All You Need' paper?"

# Execute the query
result = process_query(question)

print("\nFinal Answer:")
pprint(result)


Processing query: What is the transformer architecture introduced in the 'Attention is All You Need' paper?


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 99f4fb2b-53ba-40d1-8b2c-67746d3f9259                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a832688e-cca2-41f2-a15a-ca075b041d1e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a832688e-cca2-41f2-a15a-ca075b041d1e
       Status: Executing Task...
    └── 🤖 Agent: router
            Status: In Progress

# Agent: router
## Task: Given the question: 'What is the transformer architecture introduced in the 'Attention is All You Need' paper?', determine if it can be answered using only the provided PDF document.


...

🚀 Crew: crew
└── 📋 Task: 5110ae10-9d97-4172-a67b-7f89355c4aa7
       Status: Executing Task...
    └── 🤖 Agent: Expert Data Analyst
            Status: ✅ Completed
        └── 🔧 Used Search a PDF's content (14)

🚀 Crew: crew
└── 📋 Task: 5110ae10-9d97-4172-a67b-7f89355c4aa7
       Assigned to: Expert Data Analyst
       Status: ✅ Completed
    └── 🤖 Agent: Expert Data Analyst
            Status: ✅ Completed
        └── 🔧 Used Search a PDF's content (14)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5110ae10-9d97-4172-a67b-7f89355c4aa7                                                                     │
│  Agent: Expert Data Analyst                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 492f4384-6723-4799-8abb-6e6282433106                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
("The transformer architecture introduced in the 'Attention is All You Need' "
 'paper is a neural network model that relies entirely on self-attention '
 'mechanisms to compute representations of input and output sequences. This '
 'architecture is designed to address the limitations of traditional '
 'sequence-to-sequence models like RNNs and LSTMs by eliminating recurrent '
 'connections and instead using self-attention mechanisms to capture '
 'dependencies between input and output tokens.\n'
 '\n'
 'The transformer model consists of an encoder and a decoder, both composed of '
 'multiple layers of self-attention mechanisms and feed-forward neural '
 'networks. The self-attention mechanism allows each token in the sequence to '
 'attend to all other tokens in the sequence, capturing long-range '
 'dependencies efficiently.\n'
 '\n'
 'The key components of the transformer architecture include multi-head '
 'self-attention mechanisms, feed-forward neural networks, laye

In [37]:
with open("researcher_1.txt", "w") as text_file:
    text_file.write(result)

In [38]:
!test/run_test.sh 1

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4120511) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_01.py .                                                   [100%]

============================== 1 passed in 0.01s ===============================
Testing completed.


In [22]:
# Pick your second 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = "What is the difference between supervised and unsupervised learning?"
result = process_query(question)

print("\nFinal Answer:")
pprint(result)


Processing query: What is the difference between supervised and unsupervised learning?


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 49f67d94-c3bb-4997-b2d3-ec090318627b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 64c0ea01-c94a-4e2c-bad8-99b2ccf24941
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 64c0ea01-c94a-4e2c-bad8-99b2ccf24941
       Status: Executing Task...
    └── 🤖 Agent: router
            Status: In Progress

# Agent: router
## Task: Given the question: 'What is the difference between supervised and unsupervised learning?', determine if it can be answered using only the provided PDF document.


...

🚀 Crew: crew
├── 📋 Task: 7d82922f-eb35-4cff-953e-74b6853aa147
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (4)
└── 📋 Task: 4a0b467d-b575-46f7-974d-8ad862c7eb70
       Status: Executing Task...
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Used Read website content (4)

🚀 Crew: crew
├── 📋 Task: 7d82922f-eb35-4cff-953e-74b6853aa147
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (4)
└── 📋 Task: 4a0b467d-b575-46f7-974d-8ad862c7eb70
       Assigned to: Expert Web Scraper Agent
       Status: ✅ Completed
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Used Read website content (4)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4a0b467d-b575-46f7-974d-8ad862c7eb70                                                                     │
│  Agent: Expert Web Scraper Agent                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5ff438f2-824b-44ae-bc00-8309c87474ca                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
('Supervised learning and unsupervised learning are two main categories of '
 'machine learning approaches:\n'
 '\n'
 '1. Supervised Learning:\n'
 '- In supervised learning, the algorithm is trained on a labeled dataset, '
 'where the input data is paired with the corresponding correct output.\n'
 '- The goal of supervised learning is to learn a mapping function from input '
 'variables to output labels, enabling the algorithm to make predictions or '
 'classify new data.\n'
 '- It is typically used for tasks like classification and regression, where '
 'the algorithm learns to predict outcomes based on labeled data.\n'
 '- Supervised learning requires labeled data for training, and the algorithm '
 'aims to minimize the error between predicted and actual outputs during '
 'training.\n'
 '- Examples of supervised learning algorithms include linear regression, '
 'logistic regression, support vector machines, and neural networks.\n'
 '\n'
 '2. Unsupervised Learning:\n'
 '

In [23]:
with open("researcher_2.txt", "w") as text_file:
    text_file.write(result)

In [24]:
!test/run_test.sh 2

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4120511) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_02.py .                                                   [100%]

============================== 1 passed in 0.02s ===============================
Testing completed.


In [25]:
# Pick your third 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = " What are the key components of the Transformer architecture? "
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

Processing query:  What are the key components of the Transformer architecture? 


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 046254d6-91a4-4769-85a6-7ccead14d241                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: c009edbb-0324-4c77-9731-2806ff46dd0f
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: c009edbb-0324-4c77-9731-2806ff46dd0f
       Status: Executing Task...
    └── 🤖 Agent: router
            Status: In Progress

# Agent: router
## Task: Given the question: ' What are the key components of the Transformer architecture? ', determine if it can be answered using only the provided PDF document.


...

🚀 Crew: crew
└── 📋 Task: e2f2fd24-d34a-4bcf-8f23-2c6902834959
       Status: Executing Task...
    └── 🤖 Agent: Expert Data Analyst
            Status: ✅ Completed
        └── 🔧 Used Search a PDF's content (8)

🚀 Crew: crew
└── 📋 Task: e2f2fd24-d34a-4bcf-8f23-2c6902834959
       Assigned to: Expert Data Analyst
       Status: ✅ Completed
    └── 🤖 Agent: Expert Data Analyst
            Status: ✅ Completed
        └── 🔧 Used Search a PDF's content (8)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e2f2fd24-d34a-4bcf-8f23-2c6902834959                                                                     │
│  Agent: Expert Data Analyst                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8b284bdb-e3f9-464b-b29d-a08ada2c4400                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
('The key components of the Transformer architecture are:\n'
 '1. **Attention Mechanism**: Allows the model to weigh the importance of '
 'different parts of the input sequence when generating an output.\n'
 '2. **Encoder**: Composed of a stack of identical layers, each with two '
 'sub-layers - a multi-head self-attention mechanism and a feed-forward neural '
 'network.\n'
 '3. **Decoder**: Also composed of a stack of identical layers, with three '
 'sub-layers - a multi-head self-attention mechanism, a multi-head attention '
 "mechanism over the encoder's output, and a feed-forward neural network.\n"
 '4. **Multi-Head Attention**: Allows the model to focus on different parts of '
 'the sequence simultaneously by projecting the input into multiple subspaces '
 'and computing attention in each subspace.\n'
 '5. **Position-wise Feed-Forward Networks**: Used in both the encoder and '
 'decoder layers to apply a simple feed-forward neural network to each '
 'position separa

In [26]:
with open("researcher_3.txt", "w") as text_file:
    text_file.write(result)

In [27]:
!test/run_test.sh 3

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4120511) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_03.py .                                                   [100%]

============================== 1 passed in 0.02s ===============================
Testing completed.


In [28]:
# Pick three different appropriate questions (input query) to test the execution of the 'journalist' branch
# where the 'journalist' agent extracts an answer for the question 
# via a web search and scraping of the most relavant web page found:

# Example: "How does LORA work in large language model fine-tuning?"

# Choose a different question as input!

In [30]:
# Pick your first 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = "How does LoRA improve efficiency in fine-tuning large language models compared to full fine-tuning?"
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

Processing query: How does LoRA improve efficiency in fine-tuning large language models compared to full fine-tuning?


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 10f2e14d-8095-480d-8d66-a37cb911e7a9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1362da70-5532-4dbd-9210-87227d0847cf
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1362da70-5532-4dbd-9210-87227d0847cf
       Status: Executing Task...
    └── 🤖 Agent: router
            Status: In Progress

# Agent: router
## Task: Given the question: 'How does LoRA improve efficiency in fine-tuning large language models compared to full fine-tuning?', determine if it can be answered using only the provided PDF document.


...

🚀 Crew: crew
├── 📋 Task: ac1a09d3-8bdf-4fb7-a4b6-0c4fdda9c530
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: 5d70202b-8d5c-4407-820e-30a01bbc195f
       Status: Executing Task...
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Used Read website content (6)

🚀 Crew: crew
├── 📋 Task: ac1a09d3-8bdf-4fb7-a4b6-0c4fdda9c530
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: 5d70202b-8d5c-4407-820e-30a01bbc195f
       Assigned to: Expert Web Scraper Agent
       Status: ✅ Completed
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Used Read website content (6)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5d70202b-8d5c-4407-820e-30a01bbc195f                                                                     │
│  Agent: Expert Web Scraper Agent                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 82d07697-0393-4498-bc2d-9d8d33486602                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
('LoRA improves efficiency in fine-tuning large language models compared to '
 'full fine-tuning by being a parameter-efficient method that trains only '
 'low-rank perturbations to selected weight matrices. This means that LoRA '
 'focuses on adjusting specific parts of the pre-trained model rather than '
 'optimizing all model parameters, which can lead to faster training times and '
 'reduced computational resources required. Additionally, LoRA acts as a '
 "regularizer, helping to prevent overfitting and maintain the model's "
 'generalizability by preserving performance on tasks outside the target '
 'domain. This regularization property of LoRA can lead to improved efficiency '
 'in scenarios where the model needs to perform well on multiple related tasks '
 'without requiring extensive retraining of the entire model.')


In [31]:
with open("journalist_1.txt", "w") as text_file:
    text_file.write(result)

In [32]:
!test/run_test.sh 4

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4120511) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_04.py .                                                   [100%]

============================== 1 passed in 0.02s ===============================
Testing completed.


In [39]:
# Pick your second 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = "How are multimodal transformers evolving to handle more complex inputs in 2024?"
result = process_query(question)

print("\nFinal Answer:")
pprint(result)

Processing query: How are multimodal transformers evolving to handle more complex inputs in 2024?


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4bb2cf53-c6b4-4cd4-892b-99ebcc756ca8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f69969c2-235d-44a2-8f75-fa82cf47b3d8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f69969c2-235d-44a2-8f75-fa82cf47b3d8
       Status: Executing Task...
    └── 🤖 Agent: router
            Status: In Progress

# Agent: router
## Task: Given the question: 'How are multimodal transformers evolving to handle more complex inputs in 2024?', determine if it can be answered using only the provided PDF document.


...

🚀 Crew: crew
├── 📋 Task: 93003a49-8532-4b20-a74a-4a8d7a30150e
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (9)
└── 📋 Task: 6039490f-c16d-4b7d-9661-b069f2a12bf4
       Status: Executing Task...
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Used Read website content (9)

🚀 Crew: crew
├── 📋 Task: 93003a49-8532-4b20-a74a-4a8d7a30150e
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (9)
└── 📋 Task: 6039490f-c16d-4b7d-9661-b069f2a12bf4
       Assigned to: Expert Web Scraper Agent
       Status: ✅ Completed
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Used Read website content (9)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6039490f-c16d-4b7d-9661-b069f2a12bf4                                                                     │
│  Agent: Expert Web Scraper Agent                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5b2147ea-d31a-494a-8184-0ea5701c7f78                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
('In 2024, multimodal transformers are evolving to handle more complex inputs '
 'by integrating various modalities of data, such as text, images, videos, and '
 'audio, into a single model. These models are designed to understand and '
 'process information from different sources simultaneously, allowing for a '
 'more comprehensive analysis of the input data.\n'
 '\n'
 'Some key advancements in multimodal transformers in 2024 include:\n'
 '\n'
 '1. **Improved Fusion Mechanisms:** Multimodal transformers are incorporating '
 'advanced fusion mechanisms to effectively combine information from diverse '
 'modalities. This helps in capturing complex relationships and dependencies '
 'between different types of data.\n'
 '\n'
 '2. **Attention Mechanisms:** Enhanced attention mechanisms are being '
 'developed to enable the model to focus on relevant aspects of each modality '
 "while considering their interactions. This improves the model's ability to "
 'extract meaningful

In [40]:
with open("journalist_2.txt", "w") as text_file:
    text_file.write(result)

In [41]:
!test/run_test.sh 5

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4120511) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_05.py .                                                   [100%]

============================== 1 passed in 0.02s ===============================
Testing completed.


In [42]:
# Pick your third 'question' (replace the ... and insert your string) 
# and execute the `process_query(question)`:
question = "How do sparse attention mechanisms improve transformer efficiency at scale?"
result = process_query(question)

print("\nFinal Answer:")
pprint(result)


Processing query: How do sparse attention mechanisms improve transformer efficiency at scale?


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 59b2dfea-e2a8-440f-a748-10ca8d18be16                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b5a5ad0f-0591-4bec-b1eb-c3a6a4663a4e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b5a5ad0f-0591-4bec-b1eb-c3a6a4663a4e
       Status: Executing Task...
    └── 🤖 Agent: router
            Status: In Progress

# Agent: router
## Task: Given the question: 'How do sparse attention mechanisms improve transformer efficiency at scale?', determine if it can be answered using only the provided PDF document.


...

🚀 Crew: crew
├── 📋 Task: e5260191-3196-417e-a00e-220f9b34ced9
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (10)
└── 📋 Task: 83bca8cb-471b-426d-a4e9-73dc6bf19dc1
       Status: Executing Task...
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Using Read website content (34)

🚀 Crew: crew
├── 📋 Task: e5260191-3196-417e-a00e-220f9b34ced9
│      Assigned to: Expert Web Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Web Search Agent
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (10)
└── 📋 Task: 83bca8cb-471b-426d-a4e9-73dc6bf19dc1
       Assigned to: Expert Web Scraper Agent
       Status: ✅ Completed
    └── 🤖 Agent: Expert Web Scraper Agent
            Status: ✅ Completed
        └── 🔧 Using Read website content (34)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 83bca8cb-471b-426d-a4e9-73dc6bf19dc1                                                                     │
│  Agent: Expert Web Scraper Agent                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 33a59486-5ab1-47f3-b45a-9a505fa79e63                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
('Sparse attention mechanisms improve transformer efficiency at scale by '
 'reducing the computational complexity and memory requirements associated '
 'with self-attention mechanisms, especially when dealing with long sequences. '
 'Here are some key ways in which sparse attention mechanisms achieve this:\n'
 '\n'
 '1. **Reduced Computational Complexity**: Traditional self-attention '
 'mechanisms in transformers have a quadratic computational complexity, '
 'meaning the time taken for computation increases quadratically with the '
 'sequence length. Sparse attention mechanisms, on the other hand, enable more '
 'efficient computation by focusing only on a subset of key-value pairs '
 'relevant to each query, resulting in a linear time complexity. This '
 'reduction in computational complexity makes it more feasible to handle long '
 'sequences efficiently.\n'
 '\n'
 '2. **Memory Efficiency**: Sparse attention mechanisms help in reducing the '
 'memory footprint requir

In [44]:
with open("journalist_3.txt", "w") as text_file:
    text_file.write(result)

In [45]:
!test/run_test.sh 6

/opt/anaconda3/envs/conda-msa8700/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=4120511) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_06.py .                                                   [100%]

============================== 1 passed in 0.02s ===============================
Testing completed.


# Run all tests:

In [46]:
!test/run_test.sh

============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_01.py .                                                   [100%]

============================== 1 passed in 0.02s ===============================
============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0
rootdir: /home/hkamble2/BuildingGenerativeAIBusinessSolutions/Homework2
plugins: anyio-4.6.2
collected 1 item                                                               

test/test_hw02_02.py .                                                   [100%]

============================== 1 passed in 0.02s ===============================
============================= test session sta

# Submission Instructions:

- Submit your **6 responses** (three reponses each for the two branches listed above) as text files described above.

- Submit your **executed (!) Jupyter notebook**. See instructions below.

- **Important:** Your notebook solution needs to be executable with the Python packages listed and return correct answers with the selected queries.

# Homework submission on new IFI cluster:
- This homework is due by **2025-03-31, 2:30 PM (EDT) (Section 006)** or **2025-03-31, 6:00 PM (EDT) (Section 003)**, respectively.
- Make sure that all your programs and output files are in the exact folder as specified in the instructions.
- All file names on this system are case sensitive. Verify if you copy your work from a local computer to your home directory on the new cluster.
- **Execute the cells below to submit your assignment (6 text files and executed notebook):**

In [48]:
# Execute this cell. DO NOT EDIT THIS CONTENT!
from msa8700 import Homework2
Homework2(__session__).submit().list()

Created Homework 2.

  ____        _               _ _   
 / ___| _   _| |__  _ __ ___ (_) |_ 
 \___ \| | | | '_ \| '_ ` _ \| | __|
  ___) | |_| | |_) | | | | | | | |_ 
 |____/ \__,_|_.__/|_| |_| |_|_|\__|
                                    

Submit on behalf of: hkamble2
Submitted file: AgenticAI_RAG_Homework2_hkamble2.ipynb (1).ipynb
Submitted file: researcher_1.txt
Submitted file: researcher_2.txt
Submitted file: researcher_3.txt
Submitted file: journalist_1.txt
Submitted file: journalist_2.txt
Submitted file: journalist_3.txt
Your submission has been completed. You can re-submit your updated assignment until the deadline.

  _     _     _   _             
 | |   (_)___| |_(_)_ __   __ _ 
 | |   | / __| __| | '_ \ / _` |
 | |___| \__ \ |_| | | | | (_| |
 |_____|_|___/\__|_|_| |_|\__, |
                          |___/    

total 1220
-rw-r--r-- 1 hkamble2 hkamble2 1221797 Apr  1 00:44 AgenticAI_RAG_Homework2_hkamble2.ipynb (1).ipynb
-rw-r--r-- 1 hkamble2 hkamble2     793 Apr  1 00:3